In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
# =============================================================================
# Read Data
# =============================================================================
batch_size = 64
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [2]:
# =============================================================================
# Define Model
# =============================================================================

class Inception(nn.Module):
    def __init__(self,inp_pre):
        super(Inception, self).__init__()
        self.br1_avgp = nn.Conv2d(inp_pre, 24, kernel_size=1)
        
        self.br2 = nn.Conv2d(inp_pre, 16, kernel_size=1)

        self.br3_1 = nn.Conv2d(inp_pre, 16, kernel_size=1)
        self.br3_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.br4_1 = nn.Conv2d(inp_pre, 16, kernel_size=1)
        self.br4_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.br4_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
    def forward(self, x):
        #12*12
        br1_avgp = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)#12*12->12 Must kernel=3,pad=1(to hold dim)   
        br1_avgp = self.br1_avgp(br1_avgp)#12->12
        # => 24*12*12
#        print(np.shape(br1_avgp))dim
        br2 = self.br2(x)#12->12
        #=> 16*12*12

        br3 = self.br3_1(x)#12->12
        br3 = self.br3_2(br3)#12->12
        #=>24*12*12

        br4 = self.br4_1(x)#12->12
        br4 = self.br4_2(br4)#12->12
        br4 = self.br4_3(br4)#12->12
        #=> 24*12*12

        outputs = [br1_avgp, br2, br3, br4]
        
        #=> 88
        return torch.cat(outputs, 1)

class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)#12*12
        self.incept1 = Inception(inp_pre=10)
        
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)#Calculate
        self.incept2 = Inception(inp_pre=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)
# Size= ((N-F)/Stride)+1
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
#        print(np.shape(x))
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
#        print(np.shape(x))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return F.log_softmax(x)

model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
arLoss=[]
for epoch in range(1, 10):
    model.train()
    for index, (x, y) in enumerate(train_loader):
        x, y = Variable(x), Variable(y)
        y_pred = model(x)
        loss = F.nll_loss(y_pred, y)
        arLoss.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} Loss:{}'.format(epoch, loss.data))
plt.plot(arLoss)

/home/dominic/yes/lib/python3.6/site-packages/ipykernel_launcher.py:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 Loss:0.14907433092594147
